<a href="https://colab.research.google.com/github/hussainkazim613-dev/-A-Streamlit-Interface-For-A-Locally-Installed-LLM-/blob/main/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("🔧 Installing required packages...")
print("=" * 70)

# Install core dependencies
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate bitsandbytes
!pip install -q openai-whisper
!pip install -q unsloth
!pip install -q xformers

print("✅ Installation complete!\n")


🔧 Installing required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 53.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import whisper
import gc
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import Audio, display
import numpy as np
from scipy.io import wavfile
import time

warnings.filterwarnings('ignore')

# Check GPU availability
print("🖥️  GPU Information:")
print("=" * 70)
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  No GPU available. This pipeline requires GPU acceleration.")
print()

🖥️  GPU Information:
✅ GPU Available: Tesla T4
   Total Memory: 15.83 GB
   CUDA Version: 12.8



In [3]:
class WhisperTranscriber:
    """Handles audio transcription using OpenAI's Whisper"""

    def __init__(self, model_size="base", device="cuda"):
        """
        Initialize Whisper model

        Args:
            model_size: 'tiny', 'base', 'small', 'medium', 'large'
            device: 'cuda' or 'cpu'
        """
        print(f"🎤 Loading Whisper model ({model_size})...")
        self.device = device
        self.model = whisper.load_model(model_size, device=device)
        print(f"✅ Whisper model loaded successfully!\n")

    def transcribe(self, audio_path, language=None):
        """
        Transcribe audio file to text

        Args:
            audio_path: Path to audio file
            language: Optional language code (e.g., 'en', 'es')

        Returns:
            dict with transcription results
        """
        print(f"🎧 Transcribing audio: {audio_path}")

        start_time = time.time()
        result = self.model.transcribe(
            audio_path,
            language=language,
            fp16=torch.cuda.is_available()
        )
        elapsed = time.time() - start_time

        print(f"✅ Transcription complete! ({elapsed:.2f}s)")
        print(f"   Text: {result['text']}\n")

        return result

    def clear_memory(self):
        """Free up GPU memory"""
        del self.model
        gc.collect()
        torch.cuda.empty_cache()

In [4]:
class QuantizedReasoningModel:
    """Handles reasoning tasks using a quantized LLM"""

    def __init__(self, model_name="unsloth/llama-3-8b-Instruct-bnb-4bit", max_seq_length=2048):
        """
        Initialize quantized reasoning model

        Args:
            model_name: HuggingFace model identifier
            max_seq_length: Maximum sequence length
        """
        print(f"🧠 Loading quantized reasoning model...")
        print(f"   Model: {model_name}")

        self.max_seq_length = max_seq_length

        # Configure 4-bit quantization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

        # Load tokenizer
        print("   Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "left"

        # Load model with quantization
        print("   Loading model with 4-bit quantization...")
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True
        )

        self.model.eval()
        print(f"✅ Model loaded successfully!")
        self._print_memory_usage()
        print()

    def _print_memory_usage(self):
        """Print current GPU memory usage"""
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated(0) / 1e9
            reserved = torch.cuda.memory_reserved(0) / 1e9
            print(f"   GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

    def generate_response(self, prompt, max_new_tokens=512, temperature=0.7):
        """
        Generate reasoning response from prompt

        Args:
            prompt: Input text prompt
            max_new_tokens: Maximum tokens to generate
            temperature: Sampling temperature

        Returns:
            Generated response text
        """
        print("💭 Generating reasoning response...")

        # Format prompt for instruction-following
        formatted_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant that provides thoughtful, logical, and well-reasoned responses.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

        # Tokenize input
        inputs = self.tokenizer(
            formatted_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=self.max_seq_length - max_new_tokens,
            padding=True
        ).to(self.model.device)

        # Generate response
        start_time = time.time()
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )

        elapsed = time.time() - start_time

        # Decode response
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )

        tokens_generated = outputs.shape[1] - inputs['input_ids'].shape[1]
        print(f"✅ Response generated! ({elapsed:.2f}s, {tokens_generated} tokens)")
        print(f"   Speed: {tokens_generated/elapsed:.1f} tokens/sec\n")

        return response.strip()

    def batch_generate(self, prompts, max_new_tokens=512, temperature=0.7):
        """
        Generate responses for multiple prompts efficiently

        Args:
            prompts: List of input prompts
            max_new_tokens: Maximum tokens to generate per prompt
            temperature: Sampling temperature

        Returns:
            List of generated responses
        """
        print(f"💭 Batch generating {len(prompts)} responses...")

        # Format all prompts
        formatted_prompts = [
            f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant that provides thoughtful, logical, and well-reasoned responses.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
            for prompt in prompts
        ]

        # Tokenize batch
        inputs = self.tokenizer(
            formatted_prompts,
            return_tensors="pt",
            truncation=True,
            max_length=self.max_seq_length - max_new_tokens,
            padding=True
        ).to(self.model.device)

        # Generate responses
        start_time = time.time()
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )

        elapsed = time.time() - start_time

        # Decode all responses
        responses = []
        for i, output in enumerate(outputs):
            response = self.tokenizer.decode(
                output[inputs['input_ids'].shape[1]:],
                skip_special_tokens=True
            )
            responses.append(response.strip())

        print(f"✅ Batch generation complete! ({elapsed:.2f}s)\n")
        return responses

    def clear_memory(self):
        """Free up GPU memory"""
        del self.model
        del self.tokenizer
        gc.collect()
        torch.cuda.empty_cache()

In [5]:
class SpeechToReasoningPipeline:
    """Complete end-to-end speech-to-reasoning pipeline"""

    def __init__(self, whisper_model="base", llm_model="unsloth/llama-3-8b-Instruct-bnb-4bit"):
        """
        Initialize the complete pipeline

        Args:
            whisper_model: Whisper model size
            llm_model: Quantized LLM model name
        """
        print("🚀 Initializing Speech-to-Reasoning Pipeline")
        print("=" * 70)

        # Initialize Whisper
        self.transcriber = WhisperTranscriber(model_size=whisper_model)

        # Clear memory before loading LLM
        gc.collect()
        torch.cuda.empty_cache()

        # Initialize quantized LLM
        self.reasoning_model = QuantizedReasoningModel(model_name=llm_model)

        print("=" * 70)
        print("✅ Pipeline ready!\n")

    def process_audio(self, audio_path, max_new_tokens=512, temperature=0.7):
        """
        Process audio file end-to-end

        Args:
            audio_path: Path to audio file
            max_new_tokens: Maximum tokens for reasoning response
            temperature: Sampling temperature

        Returns:
            dict with transcription and reasoning response
        """
        print("🎯 Running End-to-End Pipeline")
        print("=" * 70)

        # Step 1: Transcribe audio
        transcription_result = self.transcriber.transcribe(audio_path)
        transcribed_text = transcription_result['text'].strip()

        # Step 2: Generate reasoning response
        reasoning_response = self.reasoning_model.generate_response(
            transcribed_text,
            max_new_tokens=max_new_tokens,
            temperature=temperature
        )

        print("=" * 70)
        print("✅ Pipeline Complete!\n")

        return {
            'audio_path': audio_path,
            'transcription': transcribed_text,
            'language': transcription_result.get('language', 'unknown'),
            'reasoning_response': reasoning_response
        }

    def display_results(self, results):
        """Display pipeline results in a formatted way"""
        print("📊 PIPELINE RESULTS")
        print("=" * 70)
        print(f"\n🎤 Transcription:")
        print(f"   Language: {results['language']}")
        print(f"   Text: {results['transcription']}")
        print(f"\n🧠 Reasoning Response:")
        print(f"   {results['reasoning_response']}")
        print("\n" + "=" * 70)

In [6]:
def create_sample_audio():
    """Create a sample audio file for demonstration"""
    print("🎵 Creating sample audio file...")

    # Create a simple tone (this is just for demo - replace with real audio)
    sample_rate = 16000
    duration = 3  # seconds
    frequency = 440  # Hz (A note)

    t = np.linspace(0, duration, int(sample_rate * duration))
    audio_data = np.sin(2 * np.pi * frequency * t) * 0.3
    audio_data = (audio_data * 32767).astype(np.int16)

    filename = "sample_audio.wav"
    wavfile.write(filename, sample_rate, audio_data)

    print(f"✅ Sample audio created: {filename}")
    print("⚠️  Note: This is a tone for testing. Upload your own audio file for real transcription.\n")

    return filename

In [7]:
if __name__ == "__main__":
    print("\n")
    print("=" * 70)
    print("  SPEECH-TO-REASONING PIPELINE DEMONSTRATION")
    print("=" * 70)
    print("\n")

    # Initialize pipeline
    pipeline = SpeechToReasoningPipeline(
        whisper_model="base",  # Options: tiny, base, small, medium, large
        llm_model="unsloth/llama-3-8b-Instruct-bnb-4bit"
    )

    # Create or use sample audio
    # For demo purposes, we'll note where you'd upload your audio
    print("📁 Audio Input Options:")
    print("   1. Upload your own audio file to Colab")
    print("   2. Use the file upload widget")
    print("   3. Mount Google Drive and use audio from there")
    print()

    # Example: Uncomment to use file upload
    # from google.colab import files
    # uploaded = files.upload()
    # audio_path = list(uploaded.keys())[0]

    # For demonstration with sample audio:
    audio_path = create_sample_audio()

    print("🔄 Processing audio through pipeline...\n")

    # Run the complete pipeline
    results = pipeline.process_audio(
        audio_path=audio_path,
        max_new_tokens=512,
        temperature=0.7
    )

    # Display results
    pipeline.display_results(results)

    print("\n💡 Usage Examples:")
    print("=" * 70)
    print("""
# Upload and process your own audio:
from google.colab import files
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
results = pipeline.process_audio(audio_path)

# Process with custom parameters:
results = pipeline.process_audio(
    audio_path="my_question.mp3",
    max_new_tokens=1024,
    temperature=0.5
)

# Access results:
print(results['transcription'])
print(results['reasoning_response'])
    """)

    print("\n✅ Pipeline demonstration complete!")
    print("   Upload your own audio files to test with real queries.")




  SPEECH-TO-REASONING PIPELINE DEMONSTRATION


🚀 Initializing Speech-to-Reasoning Pipeline
🎤 Loading Whisper model (base)...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 252MiB/s]


✅ Whisper model loaded successfully!

🧠 Loading quantized reasoning model...
   Model: unsloth/llama-3-8b-Instruct-bnb-4bit
   Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

✅ Model loaded successfully!
   GPU Memory: 6.00GB allocated, 7.46GB reserved

✅ Pipeline ready!

📁 Audio Input Options:
   1. Upload your own audio file to Colab
   2. Use the file upload widget
   3. Mount Google Drive and use audio from there

🎵 Creating sample audio file...
✅ Sample audio created: sample_audio.wav
⚠️  Note: This is a tone for testing. Upload your own audio file for real transcription.

🔄 Processing audio through pipeline...

🎯 Running End-to-End Pipeline
🎧 Transcribing audio: sample_audio.wav
✅ Transcription complete! (2.95s)
   Text: 

💭 Generating reasoning response...
✅ Response generated! (7.10s, 64 tokens)
   Speed: 9.0 tokens/sec

✅ Pipeline Complete!

📊 PIPELINE RESULTS

🎤 Transcription:
   Language: en
   Text: 

🧠 Reasoning Response:
   I'm here to help! Is there something specific you'd like assistance with or would you like me to suggest some topics we could discuss? I'm happy to engage in a conversation, provide information on a particular topic, or eve

In [8]:
def cleanup_memory():
    """Aggressive memory cleanup"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        print(f"🧹 Memory cleaned. Available: {torch.cuda.memory_allocated(0)/1e9:.2f}GB")

def download_sample_audio(url):
    """Download sample audio from URL"""
    import urllib.request
    filename = "downloaded_audio.mp3"
    urllib.request.urlretrieve(url, filename)
    return filename

print("\n📝 Additional utilities loaded:")
print("   - cleanup_memory(): Free up GPU memory")
print("   - download_sample_audio(url): Download audio from URL")
print("\n" + "=" * 70)


📝 Additional utilities loaded:
   - cleanup_memory(): Free up GPU memory
   - download_sample_audio(url): Download audio from URL



In [9]:
# Upload your audio file
from google.colab import files
print("Please upload an audio file (mp3, wav, m4a, etc.):")
uploaded = files.upload()

# Get the uploaded filename
audio_path = list(uploaded.keys())[0]

# Process it through the pipeline
results = pipeline.process_audio(
    audio_path=audio_path,
    max_new_tokens=512,
    temperature=0.7
)

# Display results
pipeline.display_results(results)

Please upload an audio file (mp3, wav, m4a, etc.):


Saving openai-fm-coral-serene.wav to openai-fm-coral-serene.wav
🎯 Running End-to-End Pipeline
🎧 Transcribing audio: openai-fm-coral-serene.wav
✅ Transcription complete! (5.66s)
   Text:  پوپ لیو چھاہر دہاں میں ایک ترکیا کا دورا کیا اور سدر اردوان پر زور دیا کہ وہ علمی تنازیات کو ختم کرانے اور دیرپا آمان کے پیام کے لئے ایک سالس کا قردار دا کرے۔ روسی سدر ولادمیر پوتن نے کہا کہ وہ یکرین پر فوجی کا روائیت تبھی روکیں گے جب کیف ان تمام علاقوں سے اپنی فوجیم وابس بلالے جن پر ماس کو داوہ کرتا ہے ورنا وہ تاقت کے زور پر ان علاقوں کو حاصل کر لے گا۔ اراغ کے قردستان کے خور مور گیس۔ فیلت پر افغانستان سے کیے گای روکت ہملے سے تیدہوار بند ہو گئی اور بجلی کا تتلکہ دا ہوا جس پر حکام نے مشتر کا تحکی قات کا حکم دیا۔ چین کے جنوب مذر بھی شہر کنمگ میں زلزلے کا پتا لگا نی والے آلات کی جانچ کرنے والی ایک ٹرین نے ریلوی مزدوروں کو تک کر ماردی جس سے دس سال میں ملک کے سب سے مہلق حادثے میں گیارا افراد حالاک ہو گئی۔ یورپی یونین نے شارا دیا ہے کہ وہ دیجتل کوانین کی خلاف ورزی پر ایلون مسک کے ایک سپلت فوم پر دو ہزار پچیس